In [2]:
import h5py
import numpy as np

datafile = h5py.File('SCNeuronModelCompetition.mat')
movie = datafile.get('trainingmovie_norm') # normalized movie
frhist = datafile.get('FRhist_tr') # firing rate histograms
movie = np.array(movie).reshape(288, 150, 128, 96, 1)

In [5]:
# this model is overfitting.. add dropout?

import keras
from keras.layers import Dense, Dropout, TimeDistributed
from keras.layers import Conv2D, MaxPool2D, Reshape

model = keras.models.Sequential()

model.add(TimeDistributed(Conv2D(4, (3, 3), activation='relu'), input_shape=movie.shape[1:]))
model.add(TimeDistributed(MaxPool2D((2, 2))))
model.add(TimeDistributed(Conv2D(4, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPool2D((2, 2))))
model.summary()
model.add(TimeDistributed(Reshape((2640,))))
model.add(TimeDistributed(Dense(50, activation='relu')))
model.add(TimeDistributed(Dense(50, activation='relu')))

model.add(Dense(frhist.shape[2], activation='softplus'))

adamopt = keras.optimizers.Adam(lr=0.001, decay=1e-7)
model.compile(optimizer=adamopt, loss='poisson')
model.summary()

# need smaller batch size to avoid running out of memory
history = model.fit(movie, frhist, epochs=200, batch_size=8, validation_split=0.2, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 150, 126, 94, 4)   40        
_________________________________________________________________
time_distributed_12 (TimeDis (None, 150, 63, 47, 4)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 150, 61, 45, 4)    148       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 150, 30, 22, 4)    0         
Total params: 188
Trainable params: 188
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 150, 126, 94, 4)   40        
_________________________________________________________________
time_distrib

ResourceExhaustedError: OOM when allocating tensor with shape[1200,126,94,4]
	 [[Node: time_distributed_11/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](time_distributed_11/Reshape, time_distributed_11/kernel/read)]]
	 [[Node: loss/mul/_135 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_770_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'time_distributed_11/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
 